In [2]:
import torch
import numpy as np
from copy import copy, deepcopy
from tqdm import tqdm

## Tug of War 2 lanes No FIFO

In [3]:
maker_cost = {
    'Marine T' : 50,
    'Baneling T' : 75,
    'Immortal T' : 200,
    'Marine B' : 50,
    'Baneling B' : 75,
    'Immortal B' : 200,
}
def get_big_A_spend_all(miner, 
              all_A_vectors = None, vector = None, move = 0):
    if all_A_vectors is None:
        all_A_vectors = list()
    if vector is None:
        vector = (0,0,0,0,0,0)
    if miner == 0:
        all_A_vectors.append(vector)
        return list(all_A_vectors)
    
    next_vector = copy(vector)
#     if miner < 50:
#         all_A_vectors.append(vector)
#         return list(all_A_vectors)
        
#     next_vector = copy(vector)
#     get_big_A_spend_all(miner - miner, all_A_vectors, next_vector)
    if miner >= maker_cost['Marine T']:
        if move <= 0:
            next_vector = (vector[0] + 1, vector[1],vector[2],
                            vector[3], vector[4], vector[5])
            get_big_A_spend_all(miner - maker_cost['Marine T'], all_A_vectors, next_vector, 0)
        if move <= 1:
            next_vector = (vector[0], vector[1],vector[2],
                            vector[3] + 1, vector[4], vector[5])
            get_big_A_spend_all(miner - maker_cost['Marine B'], all_A_vectors, next_vector, 1)
            
        if miner >= maker_cost['Baneling T']:
            if move <= 2:
                next_vector = (vector[0], vector[1] + 1,vector[2],
                                vector[3], vector[4], vector[5])
                get_big_A_spend_all(miner - maker_cost['Baneling T'], all_A_vectors, next_vector, 2)
            if move <= 3:
                next_vector = (vector[0], vector[1],vector[2],
                                vector[3], vector[4] + 1, vector[5])
                get_big_A_spend_all(miner - maker_cost['Baneling B'], all_A_vectors, next_vector, 3)
                
            if miner >= maker_cost['Immortal T']:
                if move <= 4:
                    next_vector = (vector[0], vector[1],vector[2] + 1,
                                    vector[3], vector[4], vector[5])
                    get_big_A_spend_all(miner - maker_cost['Immortal T'], all_A_vectors, next_vector, 4)
                if move <= 5:
                    next_vector = (vector[0], vector[1],vector[2],
                                    vector[3], vector[4], vector[5] + 1)
                    get_big_A_spend_all(miner - maker_cost['Immortal B'], all_A_vectors, next_vector, 5)

    return list(all_A_vectors)

In [4]:
# action_space = get_big_A_spend_all(225)
# print(len(action_space))
# for action in action_space:
#     print(action)

all_actions = []
mineral = 0
maker_cost_np = np.zeros(len(maker_cost))
for i, mc in enumerate(maker_cost.values()):
    maker_cost_np[i] = mc
for i in tqdm(range(1500 // 25 + 1)):
    all_actions.extend(get_big_A_spend_all(mineral))
    mineral += 25
    if mineral > 1500:
        break


 95%|█████████▌| 58/61 [00:03<00:00,  8.03it/s]


In [18]:
all_actions = np.array(all_actions)
pylon_action = np.zeros((all_actions.shape[0], 1))
print(all_actions.shape)
print(pylon_action.shape)
all_actions = np.hstack((all_actions, pylon_action))
print(all_actions.shape)
print(all_actions)
# print(all_actions[all_actions[:,5] == 7])

(91692, 6)
(91692, 1)
(91692, 7)
[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 2. ... 4. 4. 0.]
 [0. 0. 1. ... 4. 5. 0.]
 [0. 0. 0. ... 4. 6. 0.]]


In [21]:
key_mineral = 50
action_dict = {0: 1, 25: 1}
for i, a in tqdm(enumerate(np.array(all_actions[1:]))):
#     if key_mineral < 500:
#         print(np.sum(maker_cost_np * a))
    if np.sum(maker_cost_np * a[:-1]) != key_mineral:
        action_dict[key_mineral] = i + 1
        key_mineral += 25
action_dict[key_mineral] = i + 2
print(action_dict)
action_1500_dict = {}
all_actions_torch = torch.Tensor(all_actions)
action_1500_dict['actions'] = all_actions_torch
action_1500_dict['mineral'] = action_dict

torch.save(action_1500_dict, 'action_1500_dict_2L.pt')

91691it [00:00, 291313.34it/s]

{0: 1, 25: 1, 50: 3, 75: 5, 100: 8, 125: 12, 150: 19, 175: 25, 200: 38, 225: 50, 250: 69, 275: 91, 300: 121, 325: 153, 350: 200, 375: 248, 400: 315, 425: 387, 450: 481, 475: 583, 500: 714, 525: 854, 550: 1031, 575: 1221, 600: 1457, 625: 1709, 650: 2018, 675: 2350, 700: 2747, 725: 3175, 750: 3683, 775: 4225, 800: 4867, 825: 5551, 850: 6348, 875: 7204, 900: 8189, 925: 9241, 950: 10451, 975: 11737, 1000: 13205, 1025: 14769, 1050: 16538, 1075: 18422, 1100: 20543, 1125: 22795, 1150: 25319, 1175: 27997, 1200: 30984, 1225: 34148, 1250: 37663, 1275: 41385, 1300: 45497, 1325: 49849, 1350: 54642, 1375: 59702, 1400: 65263, 1425: 71127, 1450: 77543, 1475: 84313, 1500: 91692}


In [34]:
a = np.array([])
a = np.repeat(a.reshape(-1, 7), 0, axis = 0).reshape(-1, 7)
print(a.shape)

(0, 7)


In [3]:
        print(str(1 * 100) + "% \t",
             str(2 * 100) + "% \t",
             str(3 * 100) + "% \t")

100% 	 200% 	 300% 	
